In [1]:
import pickle
import sys
import codecs
import numpy as np
import pandas as pd
import seaborn as sns
import artm
print artm.version()

from os import path, mkdir
from datetime import datetime
%matplotlib inline
sys.path.insert(0, '..\\modules\\helpers')
from plot_helper import PlotMaker
from config_helper import ConfigPaths
from print_helper import PrintHelper
from sklearn.decomposition import PCA
from scipy.spatial import ConvexHull
from scipy.optimize import minimize

0.8.2


In [2]:
config = ConfigPaths('config.cfg')
plot_maker = PlotMaker()
printer = PrintHelper()

..\modules\helpers\config_helper.py:19: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.
  cfg.read(file_name)


In [3]:
print config.models_file_name, config.dataset_path 

Q:\\topic_modeling\\csi_science_collections.git\experiments\UCI_filtered_ngramm_trimmed_without_names\np_11_12_500\models.txt Q:\\topic_modeling\\csi_science_collections.git\..\data\postnauka\UCI_collections\UCI_filtered_ngramm_trimmed_without_names


In [10]:
def get_docs_lenths(_file_name):
    with codecs.open(_file_name, 'r', 'utf-8') as fin:
        # skip comment line
        fin.readline()  
        fin.readline()  
        fin.readline()  
        doc_len_counters = {}
        for line in fin:
            line_list = line.split(' ')
            doc_idx = float(line_list[0])
            val = float(line_list[2])
            if not doc_len_counters.has_key(doc_idx):
                doc_len_counters[doc_idx] = 0
            doc_len_counters[doc_idx] += val
    return doc_len_counters
def save_pickle_file(dists, filename):
    pickle_filename = path.join(config.experiment_path, filename)
    pickle_file = open(pickle_filename, 'wb')
    pickle.dump(dists, pickle_file)
    pickle_file.close()
def load_pickle_file(filename):
    pickle_filename = path.join(config.experiment_path, filename)
    pickle_file = open(pickle_filename, 'rb')
    p_file = pickle.load(pickle_file)
    pickle_file.close()
    return p_file
def save_model_pickle(_model_name, _model, _save=True):
    phi = _model.get_phi()
    phi = phi[(phi.T != 0).any()]
    theta = _model.get_theta()    
    saved_top_tokens = _model.score_tracker['top_tokens_score'].last_tokens
    if _save:
        save_pickle_file(phi, 'phi_{}.p'.format(_model_name))
        save_pickle_file(theta, 'theta_{}.p'.format(_model_name))
        save_pickle_file(saved_top_tokens, 'saved_top_tokens_{}.p'.format(_model_name))
    return phi, theta, saved_top_tokens
def load_model_pickle(_model_name, _distance_name):
    phi = load_pickle_file('phi_{}.p'.format(_model_name))
    theta = load_pickle_file('theta_{}.p'.format(_model_name))
    saved_top_tokens = load_pickle_file('saved_top_tokens_{}.p'.format(_model_name))
    distances = load_pickle_file('{}.p'.format(_distance_name))
    return phi, theta, saved_top_tokens, distances
def convert_to_vw(_nw, out_file):
    with codecs.open(out_file, 'w', 'utf-8') as fout:
        for idx, col in enumerate(_nw.columns):
            if idx % 100 == 0:
                print '[{}] processing column no {} of {}'.format(datetime.now(), idx, len(_nw.columns))
            fout.write(u'doc_{} |@default_class '.format(col))
            values = _nw[col]
            for idx in values.index.values:
                val = values[idx]
                if val != 0:
                    fout.write(u'{}:{} '.format(idx, val))
            fout.write(u'\n')
def create_sample_dataset(phi, theta, doc_lenths):
    ndw = phi.dot(theta)
    index = 0
    for col in ndw.columns:
        if index % 100 == 0:
            print '[{}] processing {} column'.format(datetime.now(), index)
        index += 1
        ndw[col] = ndw[col] * doc_lenths[col]
    return ndw
def create_sample_dataset_2(phi, theta, doc_lenths):
    ndw = phi.dot(theta)
    return ndw

In [5]:
doc_len_counters = get_docs_lenths(path.join(config.dataset_path, 'docword.pn.txt'))
phi1, theta1, saved_top_tokens1, distances_hellinger_model1 = load_model_pickle('model1', 'distances_hellinger_model1')
phi2, theta2, saved_top_tokens2, distances_hellinger_model2 = load_model_pickle('model2', 'distances_hellinger_model2')
phi3, theta3, saved_top_tokens3, distances_hellinger_model3 = load_model_pickle('model3', 'distances_hellinger_model3')
phi4, theta4, saved_top_tokens4, distances_hellinger_model4 = load_model_pickle('model4', 'distances_hellinger_model4')
phi5, theta5, saved_top_tokens5, distances_hellinger_model5 = load_model_pickle('model5', 'distances_hellinger_model5')

In [12]:
# ndw1 = create_sample_dataset(phi1, theta1, doc_len_counters)
ndw2 = create_sample_dataset(phi2, theta2, doc_len_counters)
# ndw3 = create_sample_dataset(phi3, theta3, doc_len_counters)
# ndw4 = create_sample_dataset(phi4, theta4, doc_len_counters)
# ndw5 = create_sample_dataset(phi5, theta5, doc_len_counters)

[2016-12-19 11:31:19.102000] processing 0 column
[2016-12-19 11:31:35.442000] processing 100 column
[2016-12-19 11:31:50.497000] processing 200 column
[2016-12-19 11:32:05.118000] processing 300 column
[2016-12-19 11:32:19.218000] processing 400 column
[2016-12-19 11:32:33.216000] processing 500 column
[2016-12-19 11:32:47.018000] processing 600 column
[2016-12-19 11:33:00.393000] processing 700 column
[2016-12-19 11:33:12.783000] processing 800 column
[2016-12-19 11:33:24.755000] processing 900 column
[2016-12-19 11:33:36.280000] processing 1000 column
[2016-12-19 11:33:47.446000] processing 1100 column
[2016-12-19 11:33:58.080000] processing 1200 column
[2016-12-19 11:34:08.280000] processing 1300 column
[2016-12-19 11:34:18.018000] processing 1400 column
[2016-12-19 11:34:27.299000] processing 1500 column
[2016-12-19 11:34:36.096000] processing 1600 column
[2016-12-19 11:34:44.507000] processing 1700 column
[2016-12-19 11:34:52.400000] processing 1800 column
[2016-12-19 11:34:59.876

In [ ]:
ndw2 = create_sample_dataset(phi2, theta2, doc_len_counters)
ndw10 = create_sample_dataset_2(phi1, theta1, doc_len_counters)
ndw20 = create_sample_dataset_2(phi2, theta2, doc_len_counters)
ndw30 = create_sample_dataset_2(phi3, theta3, doc_len_counters)
ndw40 = create_sample_dataset_2(phi4, theta4, doc_len_counters)
ndw50 = create_sample_dataset_2(phi5, theta5, doc_len_counters)

[2016-12-19 12:47:21.972000] processing 0 column


In [ ]:
# можно посмотреть на какую-нибудь колонуку-документ

In [ ]:
i = 40
col = ndw[ndw.columns[i]]
print len(col[col > 0.15]), doc_len_counters[ndw.columns[i]] 
sorted(col, reverse=True)

In [ ]:
%matplotlib inline

In [ ]:
ndws = ndw.values.flatten()
sns.distplot(ndws, bins=100)

In [ ]:
print len(ndws), len(ndws[ndws > 0.001])

In [ ]:
output_vw_path = path.join(config.home_dir, '..\\data\postnauka\\UCI_collections', 'sample_model_ndw')
if not path.exists(output_vw_path):
    mkdir(output_vw_path)
# convert_to_vw(ndw1, path.join(output_vw_path, 'pn_model1.vw'))
# convert_to_vw(ndw2, path.join(output_vw_path, 'pn_model2.vw'))
convert_to_vw(ndw3, path.join(output_vw_path, 'pn_model3.vw'))
convert_to_vw(ndw4, path.join(output_vw_path, 'pn_model4.vw'))
# convert_to_vw(ndw5, path.join(output_vw_path, 'pn_model5.vw'))

# output_vw_path = path.join(config.home_dir, '..\\data\postnauka\\UCI_collections', 'sample_model_ndw_pure')
# if not path.exists(output_vw_path):
#     mkdir(output_vw_path)
# convert_to_vw(ndw10, path.join(output_vw_path, 'pn_model1.vw'))
# convert_to_vw(ndw20, path.join(output_vw_path, 'pn_model2.vw'))
# convert_to_vw(ndw30, path.join(output_vw_path, 'pn_model3.vw'))
# convert_to_vw(ndw40, path.join(output_vw_path, 'pn_model4.vw'))
# convert_to_vw(ndw50, path.join(output_vw_path, 'pn_model5.vw'))

[2016-12-19 11:40:37.605000] processing column no 0 of 3446
[2016-12-19 11:41:20.738000] processing column no 100 of 3446
[2016-12-19 11:42:03.778000] processing column no 200 of 3446
[2016-12-19 11:42:46.976000] processing column no 300 of 3446
[2016-12-19 11:43:29.871000] processing column no 400 of 3446
[2016-12-19 11:44:12.872000] processing column no 500 of 3446
[2016-12-19 11:44:55.666000] processing column no 600 of 3446
[2016-12-19 11:45:38.851000] processing column no 700 of 3446
[2016-12-19 11:46:21.853000] processing column no 800 of 3446
[2016-12-19 11:47:05.630000] processing column no 900 of 3446
[2016-12-19 11:47:51.013000] processing column no 1000 of 3446
[2016-12-19 11:48:36.099000] processing column no 1100 of 3446
[2016-12-19 11:49:19.355000] processing column no 1200 of 3446
[2016-12-19 11:50:02.578000] processing column no 1300 of 3446
[2016-12-19 11:50:46.722000] processing column no 1400 of 3446
[2016-12-19 11:51:29.595000] processing column no 1500 of 3446
[201

In [ ]:
convert_to_vw(ndw2, path.join(output_vw_path, 'pn_model2.vw'))
output_vw_path = path.join(config.home_dir, '..\\data\postnauka\\UCI_collections', 'sample_model_ndw_pure')
if not path.exists(output_vw_path):
    mkdir(output_vw_path)
convert_to_vw(ndw10, path.join(output_vw_path, 'pn_model1.vw'))
convert_to_vw(ndw20, path.join(output_vw_path, 'pn_model2.vw'))
convert_to_vw(ndw30, path.join(output_vw_path, 'pn_model3.vw'))
convert_to_vw(ndw40, path.join(output_vw_path, 'pn_model4.vw'))
convert_to_vw(ndw50, path.join(output_vw_path, 'pn_model5.vw'))